# This notebook creates a consolidated and curated 5-year world happiness data from year 2020 to 2024.

In [1]:
import requests
import pandas as pd
from io import BytesIO
import pycountry_convert as pc

# ---------------------------
# Sourcing: Load Raw Data from URLs
# ---------------------------
files_to_download = [
    ("https://happiness-report.s3.amazonaws.com/2020/WHR20_DataForFigure2.1.xls", 2020),
    ("https://happiness-report.s3.amazonaws.com/2021/DataForFigure2.1WHR2021C2.xls", 2021),
    ("https://happiness-report.s3.amazonaws.com/2022/Appendix_2_Data_for_Figure_2.1.xls", 2022),
    ("https://happiness-report.s3.amazonaws.com/2023/DataForFigure2.1WHR2023.xls", 2023),
    ("https://happiness-report.s3.amazonaws.com/2024/DataForFigure2.1+with+sub+bars+2024.xls", 2024),
]

def fetch_excel_data(url, engine="xlrd"):
    """Fetch an Excel file from a URL and return a DataFrame."""
    response = requests.get(url)
    response.raise_for_status()  # Raise error if response is not successful
    return pd.read_excel(BytesIO(response.content), engine=engine)

raw_data = {}
for url, year in files_to_download:
    # Use "xlrd" for .xls files; for .xlsx, use "openpyxl"
    engine = "xlrd" if url.endswith("xls") else "openpyxl"
    df = fetch_excel_data(url, engine=engine)
    df["Year"] = year
    # Drop rows missing the happiness score (column might be named "Ladder score" or "Happiness score")
    if "Ladder score" in df.columns:
        df = df.dropna(subset=["Ladder score"])
    elif "Happiness score" in df.columns:
        df = df.dropna(subset=["Happiness score"])
    raw_data[year] = df

# ---------------------------
# Staging: Clean and Standardize DataFrames
# ---------------------------
COLUMN_MAPPING = {
    'Country name': 'Country',
    'Country': 'Country',
    'Ladder score': 'Happiness Score',  # Standardize to 'Happiness Score'
    'Happiness score': 'Happiness Score',
    'Logged GDP per capita': 'GDP per Capita',
    'Explained by: Log GDP per capita': 'GDP per Capita',
    'Explained by: GDP per capita': 'GDP per Capita',
    'Social support': 'Social Support',
    'Explained by: Social support': 'Social Support',
    'Healthy life expectancy': 'Healthy Life Expectancy',
    'Explained by: Healthy life expectancy': 'Healthy Life Expectancy',
    'Freedom to make life choices': 'Freedom to Make Life Choices',
    'Explained by: Freedom to make life choices': 'Freedom to Make Life Choices',
    'Generosity': 'Generosity',
    'Explained by: Generosity': 'Generosity',
    'Perceptions of corruption': 'Perceptions of Corruption',
    'Explained by: Perceptions of corruption': 'Perceptions of Corruption',
    'Year': 'Year'
}

REQUIRED_COLUMNS = [
    'Year', 'Country', 'Happiness Score', 'GDP per Capita', 
    'Social Support', 'Healthy Life Expectancy', 'Freedom to Make Life Choices', 
    'Generosity', 'Perceptions of Corruption'
]

def clean_dataframe(df):
    """Strip whitespace and trailing '*' characters from string columns."""
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip().str.rstrip('*')
    return df

staged_data = {}
for year, df in raw_data.items():
    df = df.rename(columns=COLUMN_MAPPING)
    # Remove duplicate columns if any
    df = df.loc[:, ~df.columns.duplicated()]
    df = clean_dataframe(df)
    # Retain only the required columns (if available)
    cols_to_keep = [col for col in REQUIRED_COLUMNS if col in df.columns]
    df = df[cols_to_keep].dropna(subset=cols_to_keep)
    staged_data[year] = df

# ---------------------------
# Integration: Combine Datasets and Add Continent Mapping
# ---------------------------
# Combine all staged DataFrames into one.
merged_df = pd.concat(staged_data.values(), ignore_index=True)

# Correct specific country names for consistency.
merged_df["Country"] = merged_df["Country"].replace({
    "Taiwan Province of China": "Taiwan",
    "Eswatini, Kingdom of": "Eswatini",
    "Turkiye": "Turkey",
    "Congo": "Congo (Brazzaville)"
})

# Reorder columns for readability.
final_columns = [
    "Year", "Country", "Happiness Score", "GDP per Capita", 
    "Social Support", "Healthy Life Expectancy", "Freedom to Make Life Choices", 
    "Generosity", "Perceptions of Corruption"
]
merged_df = merged_df.sort_values(by=["Year", "Country"])[final_columns]

# Define a function to map a country name to its continent using pycountry_convert.
def get_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except Exception:
        return "Unknown"

# Apply the continent mapping to add a new "Continent" column.
merged_df["Continent"] = merged_df["Country"].apply(get_continent)

# Apply manual continent updates to merged_df
manual_continent_updates = {
    "Congo (Brazzaville)": "Africa",
    "Congo (Kinshasa)": "Africa",
    "Hong Kong S.A.R. of China": "Asia",
    "Kosovo": "Europe",
    "North Cyprus": "Asia",
    "Palestinian Territories": "Asia"
}

for country, continent in manual_continent_updates.items():
    merged_df.loc[merged_df["Country"] == country, "Continent"] = continent

# Reorder columns to include the 'Continent' column.
final_columns = [
    "Year", "Country", "Continent", "Happiness Score", "GDP per Capita", 
    "Social Support", "Healthy Life Expectancy", "Freedom to Make Life Choices", 
    "Generosity", "Perceptions of Corruption"
]
merged_df = merged_df[final_columns]

print("✅ Integration step completed. The final merged dataset is stored in 'merged_df'.")
print(merged_df.head())

# ---------------------------
# Analysis: Calculate Average Continent KPIs
# ---------------------------
# Ensure KPI columns are numeric.
numeric_cols = [
    "Happiness Score", "GDP per Capita", "Social Support", "Healthy Life Expectancy", 
    "Freedom to Make Life Choices", "Generosity", "Perceptions of Corruption"
]
merged_df[numeric_cols] = merged_df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Calculate average KPIs by Year and Continent.
regional_avg = merged_df.groupby(["Year", "Continent"])[numeric_cols].mean().reset_index()

# Rename KPI columns for clarity.
regional_avg = regional_avg.rename(columns={
    "Happiness Score": "Average Continent Happiness Score",
    "GDP per Capita": "Average Continent GDP per Capita",
    "Social Support": "Average Continent Social Support",
    "Healthy Life Expectancy": "Average Continent Healthy Life Expectancy", 
    "Freedom to Make Life Choices": "Average Continent Freedom to Make Life Choices", 
    "Generosity": "Average Continent Generosity", 
    "Perceptions of Corruption": "Average Continent Perceptions of Corruption"
})

# Merge the calculated averages back into the main dataset.
merged_df = merged_df.merge(regional_avg, on=["Year", "Continent"], how="left")

print("✅ ETL pipeline completed. The final dataset is stored in 'merged_df'.")
print(merged_df.head())

# ---------------------------
# Reporting: Save Final DataFrame to CSV
# ---------------------------
output_file = "../data/processed/World_Happiness_processed_data.csv"
merged_df.to_csv(output_file, index=False)
print(f"✅ Final processed data saved to '{output_file}'.")

✅ Integration step completed. The final merged dataset is stored in 'merged_df'.
     Year      Country      Continent  Happiness Score  GDP per Capita  \
152  2020  Afghanistan           Asia           2.5669        7.462861   
104  2020      Albania         Europe           4.8827        9.417931   
99   2020      Algeria         Africa           5.0051        9.537965   
54   2020    Argentina  South America           5.9747        9.810955   
115  2020      Armenia           Asia           4.6768        9.100476   

     Social Support  Healthy Life Expectancy  Freedom to Make Life Choices  \
152        0.470367                52.590000                      0.396573   
104        0.671070                68.708138                      0.781994   
99         0.803385                65.905174                      0.466611   
54         0.900568                68.803802                      0.831132   
115        0.757479                66.750656                      0.712018   

     